In [48]:
import os, sys, json, glob
import pandas as pd
import numpy as np

# if recursion error occurs set recursion limit to higher value
#sys.setrecursionlimit(100000000)

#option1
#neighbor_lesion_id in data[(lesion['lesion_id'] != data['lesion_id']) & (data['x'].between(lesion['x']-1, lesion['x']+1)) & (data['y'].between(lesion['y']-1, lesion['y']+1)) & (data['z'].between(lesion['z']-1, lesion['z']+1)) & ((abs(lesion['x']-data['x']) + abs(lesion['y']-data['y']) + abs(lesion['z']-data['z'])) <= 1)].index
#option2
#neighbor_lesion_id in data[(lesion['lesion_id'] != data['lesion_id']) & (data['x'].between(lesion['x']-1, lesion['x']+1)) & (data['y'].between(lesion['y']-1, lesion['y']+1)) & (data['z'].between(lesion['z']-1, lesion['z']+1)) & ((abs(lesion['x']-data['x']) + abs(lesion['y']-data['y']) + abs(lesion['z']-data['z'])) == 1)].index
#option3
#neighbor_lesion_id in data[(abs(lesion['x']-data['x']) + abs(lesion['y']-data['y']) + abs(lesion['z']-data['z'])) == 1].index
#option4
#neighbor_lesion_id in data[(abs(lesion['x']-data['x']) + abs(lesion['y']-data['y']) + abs(lesion['z']-data['z'])) == 1].index

# set path, for example: path = '../results_bissenbay_assignmet2_codModByAlfredo/'
path = ''

# database
db = {}
data = object

def find_components(voxel_id, component_id):
    global data
    if data.at[voxel_id, 'visited'] == False:
        data.at[voxel_id, 'visited'] = True
        voxel = data.loc[voxel_id]
        res = [(voxel_id, (voxel['x'], voxel['y'], voxel['z']))]
        neighbor_voxels = data.at[voxel_id, 'neighbor_voxels']
        for neighbor_voxel_id in neighbor_voxels:
            res.extend(find_components(neighbor_voxel_id, component_id))
        return res
    else: return []

def fill_db(file, patient_id, image_time):
    #global db, components, component_id, data
    global db, data
    
    # preprocessing
    data = pd.read_csv(file)
    data = data[['Unnamed: 0', 'x', 'y', 'z']]
    data.columns.values[0] = 'voxel_id'
    
    # find neighbor voxels
    data['neighbor_voxels'] = ''
    for voxel_id in data.index:
        voxel = data.loc[voxel_id]
        neighbor_voxels = []
        for neighbor_voxel_id in data[(voxel['voxel_id'] != data['voxel_id']) & ((abs(voxel['x'] - data['x']) + abs(voxel['y'] - data['y']) + abs(voxel['z'] - data['z'])) == 1)].index:
            neighbor_voxels.append(neighbor_voxel_id)
        data.at[voxel_id, 'neighbor_voxels'] = neighbor_voxels
    
    # find components
    data['visited'] = False
    components = {}
    component_id = 1
    for voxel_id in data.index:
        if data.at[voxel_id, 'visited'] == False:
            found_component = find_components(voxel_id, component_id)
            component = {}
            component['voxels'] = found_component
            component['voxels'].sort()
            
            # calculate internal & external lesions
            internal_voxels = 0
            for internal_voxel_in_component in component['voxels']:
                if len(data.at[internal_voxel_in_component[0], 'neighbor_voxels']) == 6:
                    internal_voxels += 1
            
            total_voxels = len(component['voxels'])
            component['number_of_voxels_in_component'] = total_voxels
            external_voxels = total_voxels - internal_voxels
            component['number_of_internal_voxels'] = internal_voxels
            component['number_of_external_voxels'] = external_voxels
            component['percentage_of_internal_voxels'] = round((internal_voxels/total_voxels) * 100, 2)
            component['percentage_of_external_voxels'] = round((external_voxels/total_voxels) * 100, 2)
            
            # updating components
            components['component_' + str(component_id)] = component
            component_id += 1
    
    # create image data
    image = {}
    image['image'] = image_time
    image['components'] = components
    image['number_of_components'] = len(components)
    
    # add image to the database
    db[patient_id].append(image)

for file in sorted(glob.glob(os.path.join(path, '**/*.csv'), recursive=True)):
    patient_id, image_type, image_time = file[55:65].split('_')
    patient_id = 'subject_' + patient_id[1:]
    if patient_id not in db: db[patient_id] = []
    fill_db(file, patient_id, image_time)
    
def default(o):
    if isinstance(o, np.integer): return int(o)
    raise TypeError

with open('data.json', 'w') as outfile:
    json.dump(db, outfile, default=default)